Import Libraries 

In [39]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import csv

Load input csv file

In [41]:
#change input file path accordingly
df = pd.read_csv('/Users/deepaksinghal/Documents/PyhtonJupyterNotebook/Assignments/BigData/20388482.txt', sep=' ', header = None)

Removing special characters like ', ) ('

In [42]:
df[0] = df[0].str.slice(1, 7).astype(str)
df[1] = df[1].astype(str).str[:-2].astype(np.int64)
df[2] = df[2].str.slice(1, 7).astype(str)
df[3] = df[3].astype(str).str[:-1].astype(np.int64)

Checking number of rows in dataset

In [43]:
len(df.index)

1000

Tuple formation 

In [44]:
df['Set1'] = list(zip(df[0], df[1]))
df['Set2'] = list(zip(df[2], df[3]))
df = df.drop([0, 1,2, 3], axis=1)

Dividing the dataset into Two Halves

In [45]:
df1 = df.iloc[:500, :]
df2 = df.iloc[500:, :]
print(len(df1.index))
print(len(df2.index))

500
500


Mapper Function to make [key, value] as [Year, list(temp)]
)

---



In [46]:
def Map(key, value):
  dictionary = {}
  records = value.to_records(index=False)
  result = list(records)
  for i in result:
    for j in i:
      converted_num = int(j[0][0:4])
      if converted_num in dictionary.keys():
        dictionary[converted_num].append(j[1])
      else:
        dictionary[converted_num] = [j[1]]
  return dictionary

dictionary1 = Map(df1.index, df1)
dictionary2 = Map(df2.index, df2)


Sort function: add sorted entries for same year together in a single dictionary

In [47]:
dictionary = dictionary1

for key in dictionary2:
  if key in dictionary.keys():
    dictionary[key].extend(dictionary2[key])
  else:
    dictionary[key] = dictionary2[key]

Ordereddict = OrderedDict(sorted(dictionary.items()))

for key in (Ordereddict):
  print("{0}: {1}".format(key, Ordereddict[key]))

2010: [113, 115, 114, 116, 82, 36, 98, 98, 116, 88, 97, 65, 104, 119, 66, 39, 119, 109, 65, 98, 31, 105, 94, 52, 83, 85, 74, 54, 113, 99, 80, 94, 78, 51, 39, 88, 54, 37, 84, 85, 106, 113, 63, 97, 99, 86, 97, 96, 71, 105, 69, 72, 68, 56, 53, 109, 104, 114, 33, 98, 72, 52, 72, 71, 74, 84, 102, 93, 50, 111, 102, 37, 30, 94, 62, 107, 30, 34, 33, 118, 67, 56, 52, 51, 97, 41, 53, 51, 48, 116, 89, 39, 54, 52, 55, 67, 32, 79, 36, 79, 112, 81, 103, 120, 119, 103, 42, 67, 55, 115, 54, 94, 90, 87, 92, 98, 112, 79, 76, 73, 66, 34, 68, 74, 50, 104, 46, 103, 96, 102, 89, 118, 56, 95, 43, 55, 113, 81, 40, 100, 82, 79, 45, 91, 56, 88, 58, 34, 62, 66, 68, 74, 42, 91, 102, 35, 63, 53, 41, 115, 80, 57, 105, 80, 71, 66, 49, 110, 42, 61, 108, 53, 97, 37, 66, 78, 104, 77, 36, 105, 91, 37, 99, 118, 82, 62, 30, 120, 71, 90, 87, 33, 50, 43, 115, 110, 87, 54, 102, 74, 105, 41]
2011: [50, 41, 117, 58, 66, 52, 69, 101, 75, 104, 93, 108, 46, 76, 78, 73, 80, 57, 66, 73, 82, 33, 88, 70, 99, 71, 41, 37, 120, 50, 86, 

Partition function: All the months in year 2010 to 2015 are sent to Reducer1, and the others (2016 to 2020) are sent to Reducer2.


In [48]:
dictSplit1 = {}
dictSplit2 = {}
for key in Ordereddict:
  if(key <= 2015):
    dictSplit1[key] = Ordereddict[key]
  else:
    dictSplit2[key] = Ordereddict[key]

In [75]:
for key in (dictSplit1):
  print("{0}: {1}".format(key, dictSplit1[key]))

print("*********")

for key in (dictSplit2):
  print("{0}: {1}".format(key, dictSplit2[key]))

2010: [113, 115, 114, 116, 82, 36, 98, 98, 116, 88, 97, 65, 104, 119, 66, 39, 119, 109, 65, 98, 31, 105, 94, 52, 83, 85, 74, 54, 113, 99, 80, 94, 78, 51, 39, 88, 54, 37, 84, 85, 106, 113, 63, 97, 99, 86, 97, 96, 71, 105, 69, 72, 68, 56, 53, 109, 104, 114, 33, 98, 72, 52, 72, 71, 74, 84, 102, 93, 50, 111, 102, 37, 30, 94, 62, 107, 30, 34, 33, 118, 67, 56, 52, 51, 97, 41, 53, 51, 48, 116, 89, 39, 54, 52, 55, 67, 32, 79, 36, 79, 112, 81, 103, 120, 119, 103, 42, 67, 55, 115, 54, 94, 90, 87, 92, 98, 112, 79, 76, 73, 66, 34, 68, 74, 50, 104, 46, 103, 96, 102, 89, 118, 56, 95, 43, 55, 113, 81, 40, 100, 82, 79, 45, 91, 56, 88, 58, 34, 62, 66, 68, 74, 42, 91, 102, 35, 63, 53, 41, 115, 80, 57, 105, 80, 71, 66, 49, 110, 42, 61, 108, 53, 97, 37, 66, 78, 104, 77, 36, 105, 91, 37, 99, 118, 82, 62, 30, 120, 71, 90, 87, 33, 50, 43, 115, 110, 87, 54, 102, 74, 105, 41]
2011: [50, 41, 117, 58, 66, 52, 69, 101, 75, 104, 93, 108, 46, 76, 78, 73, 80, 57, 66, 73, 82, 33, 88, 70, 99, 71, 41, 37, 120, 50, 86, 

Reducer function: Collect all values belonging to the key and find the maximum temperature for the two ordered partitions.

In [49]:
def Reduce(dictionary):
  ans = {}
  for key in dictionary:
    dictionary[key].sort()
    ans[key] = dictionary[key][-1] 
  return ans

Finaldictionary1 = Reduce(dictSplit1)
Finaldictionary2 = Reduce(dictSplit2)

In [50]:
for key in (Finaldictionary1):
  print("{0}: {1}".format(key, Finaldictionary1[key]))

print("*********")

for key in (Finaldictionary2):
  print("{0}: {1}".format(key, Finaldictionary2[key]))

2010: 120
2011: 120
2012: 120
2013: 120
2014: 120
2015: 120
*********
2016: 120
2017: 120
2018: 120
2019: 120
2020: 120


Merge above two dictionaries into 1

In [51]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

dict3 = Merge(Finaldictionary1, Finaldictionary2)

for key in (dict3):
  print("{0}: {1}".format(key, dict3[key]))

2010: 120
2011: 120
2012: 120
2013: 120
2014: 120
2015: 120
2016: 120
2017: 120
2018: 120
2019: 120
2020: 120


Write final answer in a csv file

In [52]:
with open('Result.csv', 'w', newline='') as csvfile:
    header_key = ['Year', 'Max temp']
    new_val = csv.DictWriter(csvfile, fieldnames=header_key)
    new_val.writeheader()
    for new_k in dict3:
        new_val.writerow({'Year': new_k, 'Max temp': dict3[new_k]})